In [32]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import os
from keras.optimizers import SGD

np.random.seed(1000)
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), padding='same'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3),  padding='same'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(12,12,256,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.2))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(228))
model.add(Activation('softmax'))

#model.summary()



# Compile the model
opt = SGD(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

In [31]:
## Training images and labels 

import numpy as np
import pandas as pd
import cv2

NUM_CLASSES = 228
IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

def load_images(addrs_list):   
    images = np.empty((len(addrs_list), IMAGE_WIDTH, IMAGE_HEIGHT, 3), dtype=np.float32)
    for i, fpath in enumerate(addrs_list):
        img = cv2.imread(fpath, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (224, 224))
        images[i, ...] = img#.transpose(2, 0, 1) 
        if i % 1000 == 0:
            print('Loading images: {}'.format(i))
    return images

def get_multi_hot_labels(df, index_list):
    label_id = [df['labelId'][i] for i in index_list]
    
    labels_matrix = np.zeros([len(index_list), NUM_CLASSES], dtype=np.uint8())
    
    for i in range(len(label_id)):
        for j in range(len(label_id[i].split(' '))):
            row, col = i, int(label_id[i].split(' ')[j]) - 1
            labels_matrix[row][col] = 1
    
    return labels_matrix

train_df = pd.read_csv('/home/ec2-user/SageMaker/imat/train.csv')
validation_df = pd.read_csv('/home/ec2-user/SageMaker/imat/val.csv')

train_path_list = train_df['imagePath']
eval_path_list = validation_df['imagePath']

eval_data = load_images(eval_path_list)
eval_labels = get_multi_hot_labels(validation_df, list(range(validation_df.shape[0])))

Loading images: 0


In [26]:
eval_data.shape, eval_labels.shape

((1000, 224, 224, 3), (1000, 228))

In [27]:
train_paths = train_path_list
train_data = load_images(train_paths)
        
print('Loading train labels..')
train_labels = get_multi_hot_labels(train_df, list(range(validation_df.shape[0])))

Loading images: 0
Loading images: 1000
Loading images: 2000
Loading images: 3000
Loading images: 4000
Loading images: 5000
Loading images: 6000
Loading images: 7000
Loading images: 8000
Loading images: 9000
Loading train labels..


In [33]:
def main():
    
    train_iter_size = 500
    num_epochs = 100
    batch_size = 1
    eval_every_iters = 1
    np.random.seed(123)
    
    #train_steps = []
    #train_losses = []
    
    eval_steps = []
    eval_losses = []
    eval_precision = []
    eval_recall = []
    eval_meanfscore = []
    
    
    #print('Trained images so far: {}'.format(k * train_iter_size))
        
    # Randomly load training data and labels
    print('Loading train images..')
    random_indices = np.random.randint(0, train_df.shape[0], size=train_iter_size)        
    train_paths = [train_path_list[i] for i in random_indices]
    train_data = load_images(train_paths)
        
    print('Loading train labels..')
    train_labels = get_multi_hot_labels(train_df, random_indices)

    # Train the model
    model.fit(train_data, train_labels, epochs=num_epochs)
        
        
    # Garbage collection
    train_data = None
    train_labels = None
        
    
    eval_track = {'eval_steps':eval_steps, 
                  'eval_losses':eval_losses, 
                  'eval_precision':eval_precision, 
                  'eval_recall':eval_recall, 
                  'eval_meanfscore':eval_meanfscore}
    
    return eval_track

In [34]:
eval_track = main()

Loading train images..
Loading images: 0
Loading train labels..
Epoch 1/100
500/500 [==============================] - 8s 16ms/step - loss: 125.6027 - acc: 0.0660
Epoch 2/100
500/500 [==============================] - 5s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 3/100
500/500 [==============================] - 5s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 4/100
500/500 [==============================] - 5s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 5/100
500/500 [==============================] - 6s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 6/100
500/500 [==============================] - 5s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 7/100
500/500 [==============================] - 5s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 8/100
500/500 [==============================] - 5s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 9/100
500/500 [==============================] - 6s 11ms/step - loss: 127.5908 - acc: 0.0480
Epoch 10/100
500/500 [=======================

KeyboardInterrupt: 